In [2]:
import pandas as pd
df = pd.read_json('song.json', orient='records')

In [3]:
df.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [5]:
df.isnull().sum()

id                   0
song_played          0
time_played          0
user_id              0
user_sign_up_date    0
user_state           0
dtype: int64

In [25]:
firstuser=[]
state = pd.DataFrame({'count' : df.groupby( [ 'user_state', "user_sign_up_date", "user_id"] ).size()}).reset_index()
for index, row in state.iterrows():
    if index!=0:
        if state.at[index, 'user_state']!=state.at[index-1, 'user_state']:
            firstuser.append(index)
    else:
        firstuser.append(index)

In [28]:
state.iloc[firstuser].sort_values('user_sign_up_date')

,user_state,user_sign_up_date,user_id,count
0,Alabama,2015-05-01,5,21
167,Texas,2015-05-01,7,12
146,Oregon,2015-05-01,1,10
135,Ohio,2015-05-01,3,18
128,North Carolina,2015-05-01,2,18
104,New Mexico,2015-05-01,4,17
98,New Jersey,2015-05-01,6,26
149,Pennsylvania,2015-05-02,11,13
105,New York,2015-05-02,10,14
85,Minnesota,2015-05-02,8,2


In [30]:
stateuser_num = pd.DataFrame({'count' : df.groupby( [ 'user_state'] ).size()}).reset_index()

In [34]:
stateuser_num.sort_values('count', ascending=False).head(3)

,user_state,count
25,New York,469
4,California,425
35,Texas,230


In [35]:
stateuser_num.sort_values('count', ascending=True).head(3)

,user_state,count
13,Kansas,8
6,Connecticut,16
24,New Mexico,17


In [36]:
stateuser_num.to_csv('stateuser_num.csv')

In [37]:
df.to_csv('song.csv')

In [1]:
import pandas as pd
df = pd.read_csv('song.csv')

In [6]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [7]:
df.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [22]:
df['id'].unique().shape

(4000,)

In [14]:

from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf

In [15]:
count = 0
dsong={}
for song in df['song_played'].unique():
    dsong[song] = count
    count+=1


In [42]:
user_item = np.zeros((len(df['song_played'].unique()),len(df['id'].unique())))

In [21]:
count = 0
duser={}
for user in df['id'].unique():
    duser[user] = count
    count+=1

In [43]:
for index, row in df.iterrows():
    user = duser[row['id']]
    song = dsong[row['song_played']]
    user_item[song,user] = 1
        

In [132]:
song_user=[]
for i in range(4000):
    song_user.append([])
for index, row in df.iterrows():
    song_user[dsong[row['song_played']]].append(duser[row['id']])
    

In [33]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection / union)


In [129]:
from scipy import spatial
similar_user=[]
for i in range(4000):
    tempitem = [0,0,0,0,0]
    tempscore = [0,0,0,0,0]
    for j in range(4000):
        if j!=i:
            score = 1 - jaccard_similarity(song_user[i],song_user[j])
            p = 4
            while p>=0 and score > tempscore[p]:
                p-=1 
            tempscore.insert(p+1, score)
            tempitem.insert(p+1, j)
            tempscore.pop()
            tempitem.pop()
    similar_user.append(tempitem)   

In [88]:
jaccard_similarity(song_user[1],song_user[10])

0.0

In [104]:
for j in range(100):
    for i in range(100):
        if len(list((set(song_user[j]).intersection(song_user[i]))))!=0 and i!=j:
            print([j,i])

In [135]:
song_fre={}
for i in range(len(song_user)):
    if len(song_user[i]) not in song_fre.keys():
        song_fre[len(song_user[i])] = []
    song_fre[len(song_user[i])].append(i)
    

In [113]:
for q in range(100):
    for p in range(100):
        for i in song_user[q]:
            for j in song_user[p]:
                if i==j and p!=q:
                    print(p)

In [127]:
totalPeople = 0
for i in range(100):
    totalPeople += len(song_user[i])
print(totalPeople)

4000


In [139]:
song_fre.keys()

dict_keys([120, 99, 163, 6, 8, 11, 161, 4, 321, 184, 35, 10, 279, 408, 18, 165, 127, 170, 41, 113, 69, 9, 5, 66, 68, 40, 79, 2, 36, 16, 28, 39, 24, 29, 61, 116, 57, 17, 20, 31, 33, 30, 14, 19, 22, 7, 27, 21, 32, 3, 0])

In [140]:
song_fre[321]

[8]

In [141]:
song_fre[408]

[13]

In [142]:
song_fre[279]

[12]

In [143]:
song_fre[184]

[9]

In [ ]:
#location recommandation, user-item in future